# Spring 2020 -> INSY 5336 - 001 -> Python Programming
## Project
### Name: Phanikrishna Karanam (UTA ID# 1001851985)

#### __Project__:  
__Summary__: This project is about scraping data from CNN and Yahoo finance websites. The program gets the most active tickers from CNN website and details like open price, previous close price and volume are fetched from yahoo finance website for those tickers. All of these details are written into a text file in CSV format and also stored into a SQLite database. 

##### __Execution Instructions__:
1. Run the cell below   
2. Wait for 10-20 seconds for program execution to complete

Note: If CNN or Yahoo Finance websites are down, then please try again 

##### __Expected Results__:
 - Once user executes the program:  
     - Program execution started message is displayed 
     - Program execution ended message is displayed after 10-20 seconds along with elapsed time details
     - Active ticker details (Ticker, Open price, Previous close price, Volume) are written into stocks.txt file in csv format 
         - If file exists, then a new file with same name is written 
     - The same details as in the stocks file are also written into SQLite database   
         - If StocksTable already exists, then its dropped and new one is created
 - Error message is displayed for below: 
     - If any of the websites are down

Note: 
 - Validate the output stocks.txt file and StocksTable (within StocksDatabase.sqlite database) using SQLite DBbrowser 
 - Notepad file stock.txt and StockDatabase.sqlite database will be created in the same directory from where Python code file
   is being executed from 

##### __Algorithm__: 
` 1. Import libraries required by the program `  
` 2. Display the program execution started message, collect the system time and set website up flag to True `  
` 3. Open the CNN url and get the url handle `  
` 4. If any error during url open, then display error message and exit the program `   
` 5. If no error, then continue processing by reading the url handle and decoding the html message `  
` 6. Using regex extract the text pertaining to Most Active tickers only, so that search criteria length is limited `  
` 7. Extract all the most active tickers using regex by looking for text that appears before the ticker value `  
` 8. Initialize two lists, one for writing into file and other for writing to database `   
` 9. Loop thru all the active tickers and perform below:  
     9.1 Initialize two lists, one for storing the html text content of each ticker and other for details about required
         individual ticker values  
     9.2 Open the Yahoo finance url for ticker  
         9.2.1 If website is down, then set the website up flag to false and exit program execution 
         9.2.2 If no error, then continue processing 
     9.3 Read the url handle and decoding the html message
     9.4 Using regex extract the text pertaining upto volume details, so that search criteria length is limited
     9.5 Write the ticker name into the ticker details list as first entry 
     9.6 Extract the open value from text created in 9.4. Convert the value into float and write into detail list
     9.7 Extract the previous close value from text created in 9.4. Convert the value into float and write into detail list
     9.8 Extract the volume from text created in 9.4. Remove the commas in value text, convert into integer and write the value 
         into detail list
     9.9 Append this detail list having details about one ticker into main list as one item 
     9.10 Convert this detail list also into a string using join function and delimit each list item using a comma. Also add a 
          new line character at the end of the string. Use this string and append into file write list `  
` 10. If website flag is set to True:  
     10.1 Open the stock.txt file in write mode and obtain the file handle 
     10.2 Using writelines function and the file write list created in 9.10, write into the file 
     10.3 Close the file 
     10.4 Connect to SQLite StockDatabase and connect to the cursor 
     10.5 Drop the StocksTable if it exists 
     10.6 Create the StocksTable with columns (Ticker TEXT, OpenPrice REAL, PrevClose REAL, Volume INTEGER)
     10.7 Using executemany command insert the records into table at once by passing the main list created in step 9.9
     10.8 Close the cursor 
     10.9 Commit the insert performed into database
     10.10 Close the connection
     10.11 Print the elapsed time taken for program execution
           10.11.1 Subtract the current time from time captured in step 2, to get the elapsed time `  
` 11. Display program execution ended message `  

In [1]:
# Start of Program

# Import required libraries for program execution 
import urllib.request, urllib.parse, urllib.error
import re 
import sqlite3
import time

start_time = time.time()                                     # Get program execution start time               
print('Program execution started')                  
website_up = True                                            # Flag to check if website is up/down. Set to True by default

try:                                                         # Open CNN website and get url handle  
    cnn_fhand = urllib.request.urlopen('https://money.cnn.com/data/hotstocks/')
except:  
    print('CNN website is currently down. Please try again') # When website is down, display error message   
else: 
    cnn_html_str = cnn_fhand.read().decode()                 # If website is up, then read html and decode into text      

# Below code will extract html text for Most Actives tickers part alone, so that search string length is limited
# Then active tickers are extracted into a list using regex by searching for string appearing before ticker value
    active_list = re.findall('<h3>Most Actives</h3>.*?</table>', cnn_html_str, re.DOTALL)
    active_tickers = re.findall('class="wsod_symbol">([^ <]*)', active_list[0])

# Each of the tickers extracted above will be passed into below Yahoo Finance url, to extract additional values 
    yf_url  = 'https://finance.yahoo.com/quote/{tick}?p={tick}&.tsrc=fin-srch-v1'
    main_list = []                                           # List for storing data to be written into database   
    file_write_list = []                                     # List for storing data to be written into file

    for ticker in active_tickers:                            # Loop thru each ticker value & perform below processing
        yf_html_list = []                                    # List to store limited html content 
        temp_list = []                                       # List to store details of individual active ticker  

        try: 
            yf_fhand = urllib.request.urlopen(yf_url.format(tick = ticker))    # Open Yahoo Finance url for each ticker 
        except: 
            print('Yahoo Finance website is currently down. Please try again') # Error message when website is down
            website_up = False                               # Set flag to False   
            break                                            # Break from For loop when error
        else: 
            yf_html_str = yf_fhand.read().decode()           # If website is up, then read html and decode into text 
            
# Below code will extract html text from Prev Close until Average tag, so that search string length is limited            
            yf_html_list = re.findall('PREV_CLOSE-value.*?Avg', yf_html_str, re.DOTALL)

            temp_list.append(ticker)                         # Add ticker value into temp list as first element 

# Below code will extract values for open, close and volume tags using REGEX commands 
            temp = re.findall('OPEN-value.*?([^ >]*)</span></td></tr>',yf_html_list[0], re.DOTALL)
            temp_list.append(float(temp[0]))                 # Convert the value into float and append into temp list 

            temp = re.findall('PREV_CLOSE-value.*?([^ >]*)</span></td></tr>',yf_html_list[0], re.DOTALL)
            temp_list.append(float(temp[0]))                 # Convert the value into float and append into temp list

            temp = re.findall('TD_VOLUME-value.*?([^ >]*)</span></td></tr>',yf_html_list[0], re.DOTALL)
            temp_list.append(int(temp[0].replace(',','')))   # Remove commas in text, convert to INT, append into temp list 

            main_list.append(temp_list)                      # Append entry created for individual ticker into main list
            file_write_list.append(((',').join(map(str,temp_list))) + '\n') # Create CSV format, add \n character at end

    if website_up:                                           # If website is up, then perform below processing 
        fhand = open('stocks.txt','w')                       # Open stocks file in write mode
        fhand.writelines(file_write_list)                    # Write all elements in the file list into output file  
        fhand.close()                                        # Close output file 

        conn = sqlite3.connect('StocksDatabase.sqlite')      # Connect to StocksDatabase sqlite database
        cur = conn.cursor()                                  # Open connection to cursor and get cursor handle  
        
        column_names = '(Ticker TEXT, OpenPrice REAL, PrevClose REAL, Volume INTEGER)' # Column names to be defined & Type
        
        cur.execute('DROP TABLE IF EXISTS StocksTable')      # Drop StocksTable if already present   
        cur.execute('CREATE TABLE StocksTable' + column_names) # Create StocksTable with column names and type as defined
        
        cur.executemany('INSERT INTO StocksTable VALUES (?,?,?,?);', main_list) # Insert all records at once from main list
        cur.close()                                          # Close the cursor
        
        conn.commit()                                        # Commit the record inserts into database
        conn.close()                                         # Close the connection to database  
                                                             # Subtract current time from start time & display elapsed time
        print('Execution elapsed time (HH:MM:SS):', time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time)))
        
print('Program execution ended')                                     

# End of Program

Program execution started
Execution elapsed time (HH:MM:SS): 00:00:11
Program execution ended
